In [ ]:
%load_ext autoreload
%autoreload 2

from gomat_markup_opt.data.data_preparation import (
    load_and_prepare_inventory_data,
    load_and_prepare_quote_lost_data,
    load_and_prepare_quote_order_data,
)
from gomat_markup_opt.preprocessing.data_preprocessing import (
    cleanup_and_filter_inventory_data,
    cleanup_and_filter_quote_and_order_data,
)

import os
os.environ["DATA_DIR"] = "/Users/mathieu.larouche.dube/sources/gomaterials/gomat-markup-optimization/data"

prepared_quote_order_data = load_and_prepare_quote_order_data()
prepared_quote_lost_data = load_and_prepare_quote_lost_data()
prepared_inventory_data = load_and_prepare_inventory_data()

cleaned_quote_data = cleanup_and_filter_quote_and_order_data(prepared_quote_order_data, prepared_quote_lost_data)
cleaned_inventory_data = cleanup_and_filter_inventory_data(prepared_inventory_data)


In [ ]:
import pandas as pd

quote_lat_lon_pairs = cleaned_quote_data[['latitude', 'longitude']].drop_duplicates()
inventory_lat_lon_pairs = cleaned_inventory_data[['latitude', 'longitude']].drop_duplicates()

# drop nan
quote_lat_lon_pairs = quote_lat_lon_pairs.dropna()
inventory_lat_lon_pairs = inventory_lat_lon_pairs.dropna()

all_combinations = []
for _, quote_row in quote_lat_lon_pairs.iterrows():
    for _, inventory_row in inventory_lat_lon_pairs.iterrows():
        all_combinations.append({
            'quote_latitude': quote_row['latitude'],
            'quote_longitude': quote_row['longitude'],
            'inventory_latitude': inventory_row['latitude'],
            'inventory_longitude': inventory_row['longitude']
        })

all_combinations_df = pd.DataFrame(all_combinations)
print(len(all_combinations_df))




In [ ]:
from haversine import haversine, Unit

from tqdm import tqdm
tqdm.pandas()

all_combinations_df['distance'] = all_combinations_df.progress_apply(
    lambda row: haversine(
        (row['quote_latitude'], row['quote_longitude']),
        (row['inventory_latitude'], row['inventory_longitude']),
        unit=Unit.KILOMETERS
    ),
    axis=1
)


# Filter out combinations with distance greater than 500 meters
filtered_combinations_df = all_combinations_df[all_combinations_df['distance'] <= 400]
print(len(filtered_combinations_df))


In [ ]:
print(all_combinations_df.head())